# Time Series

## 1. Importing libraries and files

In [1]:
import numpy as np 
import pandas as pd

In [2]:
df_weekly = pd.read_csv('./Data_for_Power_BI/weekly_data_for_power_bi.csv')

## 2. Feature Engineering

In [ ]:
# Lags (Past values)
# Previous week sales (t-1)
df_weekly['lag_1_sale'] = df_weekly.groupby(['item', 'store_code'])['sale'].shift(1)
df_weekly['lag_1_sellprice'] = df_weekly.groupby(['item', 'store_code'])['sell_price'].shift(1)

# Sales 4 weeks ago (approx 1 month)
df_weekly['lag_4_sale'] = df_weekly.groupby(['item', 'store_code'])['sale'].shift(4)
df_weekly['lag_4_sellprice'] = df_weekly.groupby(['item', 'store_code'])['sell_price'].shift(4)

# Sales 52 weeks ago (approx 1 year)
df_weekly['lag_52_sale'] = df_weekly.groupby(['item', 'store_code'])['sale'].shift(52)
df_weekly['lag_52_sellprice'] = df_weekly.groupby(['item', 'store_code'])['sell_price'].shift(52)

# Variation / Detrending (Target for stationary models)
# "Diff": Absolute difference (Current - Previous)
df_weekly['sales_diff'] = df_weekly.groupby(['item', 'store_code'])['sale'].diff(1)
df_weekly['price_diff'] = df_weekly.groupby(['item', 'store_code'])['sell_price'].diff(1)

# Rolling Features (Smoothing for Power BI visualization)
# 4-week moving average (shows trend without noise)
df_weekly['rolling_mean_4w_sale'] = df_weekly.groupby(['item', 'store_code'])['sale'].transform(lambda x: x.rolling(window=4).mean())
df_weekly['rolling_mean_4w_sellprice'] = df_weekly.groupby(['item', 'store_code'])['sell_price'].transform(lambda x: x.rolling(window=4).mean())


In [ ]:
df_calen